In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Step 1: 读取数据
df = pd.read_excel(r'E:\大三\大三上\Optimization in Finance\Final Project\Portefolio\Half Year\Prediction.xlsx')

# Step 2: 转换日期
df['TradingDate'] = pd.to_datetime(df['TradingDate'])

# Step 3: 生成“半年期”标识和连续序号
# 定义半年期：1-6月为 H1，7-12月为 H2
df['HalfYear'] = df['TradingDate'].dt.month.apply(lambda m: 1 if m <= 6 else 2)
df['Year'] = df['TradingDate'].dt.year

# 生成半年期字符串（可选，用于查看）
df['HalfYear_Label'] = df['Year'].astype(str) + '-H' + df['HalfYear'].astype(str)

# 生成连续的半年期序号（假设 2023-H1 是第 1 期）
# 每年有 2 个半年期，所以：(Year - 2023) * 2 + HalfYear
df['Period_Index'] = (df['Year'] - 2023) * 2 + df['HalfYear']

print("✅ 已生成 Period_Index (2023-H1 → 1, 2023-H2 → 2, 2024-H1 → 3, ...)")

# Step 4: 数据类型处理
df['Symbol'] = df['Symbol'].astype(str)
df['Predicted_ClosePrice'] = pd.to_numeric(df['Predicted_ClosePrice'], errors='coerce')

# Step 5: 每只股票每个半年期只保留最后一个交易日（即半年期末）
df = df.sort_values(['Symbol', 'TradingDate'])
df = df.drop_duplicates(subset=['Symbol', 'Period_Index'], keep='last')

# Step 6: 按股票分组计算半年期收益率
all_returns = []

for symbol in df['Symbol'].unique():
    stock_df = df[df['Symbol'] == symbol].copy()
    stock_df = stock_df.sort_values('Period_Index')
    
    # 计算半年期收益率（pct_change）
    returns = stock_df['Predicted_ClosePrice'].pct_change()

    result = pd.DataFrame({
        'Symbol': symbol,
        'Period_Index': stock_df['Period_Index'],
        'Semiannual_Return': returns.values
    })
    all_returns.append(result)

# 合并结果
final_df = pd.concat(all_returns, ignore_index=True)

# 可选：添加标签便于阅读
period_map = dict(zip(df['Period_Index'], df['HalfYear_Label']))
final_df['Period_Label'] = final_df['Period_Index'].map(period_map)

# 保存到 Excel
final_df.to_excel('修正训练数据_半年收益率.xlsx', index=False)

print("✅ 完成！已保存半年收益率到 '修正训练数据_半年收益率.xlsx'")

✅ 已生成 Period_Index (2023-H1 → 1, 2023-H2 → 2, 2024-H1 → 3, ...)
✅ 完成！已保存半年收益率到 '修正训练数据_半年收益率.xlsx'


In [3]:

# Step 1: 读取原始数据
df = pd.read_excel('修正训练数据_半年收益率.xlsx')  # 替换为你的文件名

# Step 2: 确保数据类型正确
df['Symbol'] = df['Symbol'].astype(str)
df['Index'] = df['Index'].astype(int)

# Step 3: 按 Symbol 和 Season_Index 重置索引，确保唯一性
df = df.drop_duplicates(subset=['Symbol', 'Index'], keep='last')

# Step 4: 转为宽格式（pivot）
wide_df = df.pivot(index='Index', columns='Symbol', values='Return')

# Step 5: 重置索引，使 Month_Index 成为普通列
wide_df = wide_df.reset_index()

# Step 6: 保存到新 Excel 文件
wide_df.to_excel('宽泛修正训练数据.xlsx', index=False)

print("✅ 宽格式数据已保存到 宽泛修正训练数据.xlsx")

✅ 宽格式数据已保存到 宽泛修正训练数据.xlsx


In [9]:
# 1. 读取 Excel 文件
file_path = "宽泛修正训练数据.xlsx"
df = pd.read_excel(file_path, header=None)  # 无标题行

# 2. 提取收益率数据：去掉第一列（时间点），只保留股票收益率
returns_df = df.iloc[1:, 1:]  # 所有行，从第2列开始

# 3. 检查形状
print("数据形状（时间点 × 股票数）:", returns_df.shape)

# 4. 计算样本协方差矩阵（按列：每列是一只股票）
cov_matrix = returns_df.cov()  # 默认 ddof=1，即样本协方差

print("协方差矩阵形状:", cov_matrix.shape)
print("协方差矩阵前5行5列:\n", cov_matrix.iloc[:5, :5])

# 5. （可选）保存协方差矩阵回 Excel
cov_matrix.to_excel("协方差矩阵_499x499.xlsx")
print("✅ 协方差矩阵已保存为 '协方差矩阵_499x499.xlsx'")

数据形状（时间点 × 股票数）: (7, 499)
协方差矩阵形状: (499, 499)
协方差矩阵前5行5列:
           1         2         3         4         5
1  0.011237  0.005690  0.009649  0.002781  0.002222
2  0.005690  0.018076  0.011021  0.002251  0.001063
3  0.009649  0.011021  0.023105  0.009146  0.005307
4  0.002781  0.002251  0.009146  0.010645 -0.001289
5  0.002222  0.001063  0.005307 -0.001289  0.004729
✅ 协方差矩阵已保存为 '协方差矩阵_499x499.xlsx'
